In [63]:
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import numpy as np

In [296]:
def get_info(link):

    #information :
    title = driver.find_element(by = 'xpath', value = "//h1[@class='sc-e6f263fc-1 sc-842a8720-1 bwGoop dBdZmZ']")
    title = title.text

    # 1 'Toutes les informations'
    #Ouverture de la page 'Toutes les informations'

    #element_div = driver.find_element(by='xpath', value= "//div[@class='sc-47761b26-0 hYplax']")
    #print(element_div.text)
    element_a = driver.find_elements(by='xpath', value= "//a[@data-testid='link' and @class = 'sc-e6f263fc-0 sc-a0949da7-0 iZcnfH cYbaKn']")
    for i in element_a:  # methode particuliere car quand je prends element_div et cherchant dans les a, il selectionne Explorer qui n'appartient pas a element_div (mystere)
        if i.text == 'Toutes les informations':
            element_a = i
    element_a.click()
    sleep(1)

    #Nouvelle page 
    info_div = driver.find_element(by='xpath', value="//div[@class='sc-e6f263fc-0 sc-47761b26-1 iZcnfH dIowGJ']").text
    info = info_div.split('\n')
    for i in info : 
        if 'Genre' in i:
            genre = i
        if 'Développeur' in i :
            developpeur = i
        if 'Éditeur' in i:
            editeur = i
        if 'Date de sortie (France)' in i:
            date_sortie_fr = i
        if 'Sortie sur' in i :
            console = i

    #retour en arriere
    driver.back()
    sleep(2)
    # 2 - recuperer les notes 
    try : 
        note_global = driver.find_element(by='xpath', value="//div[@class='sc-8251ce8c-5 fdoSxm' and @data-testid='Rating']").text

        #ouverture du POP UP
   
        driver.find_element(by='xpath', value="//div[@class='sc-842a8720-8 ewTuTa']").click()
        sleep(3)

        note_div = driver.find_elements(by='xpath', value="//p[@class='sc-e6f263fc-0 sc-c741f173-4 GItpw jiNQTI' and @data-testid='value-vertical-bar']")
        nb_notes = []
        for note in note_div: 
            nb_notes.append(note.text)

        note_1, note_2, note_3, note_4, note_5, note_6, note_7, note_8, note_9, note_10 = nb_notes
        

        #Fermeture du POP Up
        driver.find_element(by='xpath', value="//div[@data-testid='modal-cross' and @class='sc-6d06c511-3 bpjfzD']").click()

    except: 
        note_global = np.nan
        liste_nans = [np.nan] * 10
        note_1, note_2, note_3, note_4, note_5, note_6, note_7, note_8, note_9, note_10 = liste_nans

    liste = [title,genre,developpeur,editeur,date_sortie_fr,console,note_global,note_1, note_2, note_3, note_4, note_5, note_6, note_7, note_8, note_9, note_10,link]

    return (liste)

In [286]:
def jeux_page(elements) : 
    df = pd.DataFrame(columns = ["Nom", "Genre", "Developpeur", "Editeur" , "Date", "Console","Note_Site",
                                 "Nb_note1","Nb_note2","Nb_note3","Nb_note4","Nb_note5","Nb_note6","Nb_note7","Nb_note8","Nb_note9","Nb_note10","link"])
    for element in elements :
        link = element.get_attribute("href")
        if '/jeuvideo/grand_theft_auto' in link.lower():
            webdriver.ActionChains(driver).key_down(Keys.COMMAND).click(element).key_up(Keys.COMMAND).perform()
            sleep(1)
            tab = driver.window_handles
            search_page = tab[0]
            gta_page = tab[1]
            driver.switch_to.window(gta_page)
            info_jeu = get_info(link)
            driver.close()
            driver.switch_to.window(search_page)
            a = pd.DataFrame([info_jeu],columns = ["Nom", "Genre", "Developpeur", "Editeur" , "Date", "Console","Note_Site",
                                                    "Nb_note1","Nb_note2","Nb_note3","Nb_note4","Nb_note5","Nb_note6","Nb_note7","Nb_note8","Nb_note9","Nb_note10","link"])
            df = pd.concat([df,a])
    return(df)


In [297]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.senscritique.com/search?query=grand+theft+auto&page=1')
sleep(2)
#accepter les cookies
webelement = driver.find_element(by='id',value = 'didomi-notice-agree-button')
webelement.click()

data_gta = pd.DataFrame(columns = ["Nom", "Genre", "Developpeur", "Editeur" , "Date", "Console","Note_Site",
                                                    "Nb_note1","Nb_note2","Nb_note3","Nb_note4","Nb_note5","Nb_note6","Nb_note7","Nb_note8","Nb_note9","Nb_note10","link"])
base = 'https://www.senscritique.com/search?query=grand+theft+auto&page='

#recuperation data
for i in range(2,4):
    elements = driver.find_elements(by = 'xpath', value = "//a[@class='sc-e6f263fc-0 sc-a0949da7-1 cTWuvs eGjRhz' and @data-testid='product-explorer-title']")
    data = jeux_page(elements)
    data_gta = pd.concat([data_gta,data])
    driver.get(base+str(i))
    sleep(2)

driver.close()
display(data_gta)

,Nom,Genre,Developpeur,Editeur,Date,Console,Note_Site,Nb_note1,Nb_note2,Nb_note3,Nb_note4,Nb_note5,Nb_note6,Nb_note7,Nb_note8,Nb_note9,Nb_note10,link
0,Grand Theft Auto V,Genre : Action-Aventure,Développeur : Rockstar North,"Éditeurs : Rockstar Games, Take-Two Interactiv...",Date de sortie (France) : 17 septembre 2013,"Sortie sur : PlayStation 4, PlayStation 3, Xbo...",8,3K,1.6K,1.8K,2.2K,3.8K,5.7K,11.2K,18.3K,16.1K,19.3K,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto IV,Genre : Action-Aventure,Développeur : Rockstar North,"Éditeurs : Rockstar Games, Take-Two Interactiv...",Date de sortie (France) : 29 avril 2008,"Sortie sur : PlayStation 3, Xbox 360, PC",7.4,953,486,639,1K,2.1K,4.2K,8.6K,10K,5.5K,4.8K,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto: San Andreas,Genre : Action-Aventure,Développeur : Rockstar North,Éditeur : Rockstar Games,Date de sortie (France) : 29 octobre 2004,"Sortie sur : PlayStation 2, PlayStation 3, Pla...",8,379,204,282,360,781,1.8K,4.7K,9.2K,7.3K,5.9K,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto: Vice City,Genre : Action-Aventure,Développeur : Rockstar North,Éditeur : Rockstar Games,Date de sortie (France) : 8 novembre 2002,"Sortie sur : PlayStation 2, PC, Xbox, Mac, iPa...",7.8,361,171,230,310,766,1.9K,5.4K,7.8K,4.6K,3.1K,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto III,Genre : Action-Aventure,Développeur : DMA Design,Éditeur : Rockstar Games,Date de sortie (France) : 26 octobre 2001,"Sortie sur : PlayStation 2, PlayStation 4, Xbo...",7.4,34,30,73,140,454,1.5K,3.9K,3.9K,1.6K,684,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto: San Andreas,Genre : Action-Aventure,Développeur : Rockstar North,Éditeur : Rockstar Games,Date de sortie (France) : 12 décembre 2013,"Sortie sur : iPhone, iPad, Android",7.9,231,97,129,158,292,511,1K,1.8K,1.5K,2K,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto IV: The Lost and Damned,"Genres : Action, Aventure",Développeur : Rockstar Games,Éditeur : Rockstar Games,Date de sortie (France) : 17 février 2009,"Sortie sur : PlayStation 3, Xbox 360, PC",7.2,16,13,36,68,220,676,1.5K,1.2K,409,176,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto IV: The Ballad of Gay Tony,"Genres : Action, Aventure",Développeur : Rockstar Games,Éditeur : Rockstar Games,Date de sortie (France) : 29 octobre 2009,"Sortie sur : PC, Xbox 360, PlayStation 3",7.6,11,6,19,46,92,294,998,1.3K,498,168,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto 2,Genre : Action,Développeur : DMA Design,Éditeur : Rockstar Games,Date de sortie (France) : 22 octobre 1999,"Sortie sur : PC, PlayStation, Dreamcast",6.7,12,13,57,121,338,861,1.3K,685,256,127,https://www.senscritique.com/jeuvideo/grand_th...
0,Grand Theft Auto,Genre : Action,Développeur : DMA Design,"Éditeurs : Take-Two Interactive Software, BMG ...",Date de sortie (France) : 1 octobre 1997,"Sortie sur : PC, PlayStation, Game Boy Color, ...",6.8,15,24,49,95,252,656,907,677,278,138,https://www.senscritique.com/jeuvideo/grand_th...


In [300]:
data_gta.reset_index(drop=True)
data_gta.shape

data_gta.info()
data_gta.to_csv('gta_sc_brut.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 0 to 0
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nom          23 non-null     object
 1   Genre        23 non-null     object
 2   Developpeur  23 non-null     object
 3   Editeur      23 non-null     object
 4   Date         23 non-null     object
 5   Console      23 non-null     object
 6   Note_Site    22 non-null     object
 7   Nb_note1     22 non-null     object
 8   Nb_note2     22 non-null     object
 9   Nb_note3     22 non-null     object
 10  Nb_note4     22 non-null     object
 11  Nb_note5     22 non-null     object
 12  Nb_note6     22 non-null     object
 13  Nb_note7     22 non-null     object
 14  Nb_note8     22 non-null     object
 15  Nb_note9     22 non-null     object
 16  Nb_note10    22 non-null     object
 17  link         23 non-null     object
dtypes: object(18)
memory usage: 3.4+ KB


In [352]:
# nettoyage datadrame 

df_gta = pd.DataFrame()

df_gta['Nom'] = data_gta['Nom']

df_gta['Genre'] = data_gta['Genre'].apply(lambda x : x[8:].strip())

df_gta['Developpeur'] = data_gta['Developpeur'].apply(lambda x : x[14:].strip())

df_gta['Editeur'] = data_gta['Editeur'].apply(lambda x : x[10:].strip())

df_gta['Date'] = data_gta['Date'].apply(lambda x : x[25:].strip())

df_gta['Console'] = data_gta['Console'].apply(lambda x : x[13:].strip())

df_gta['Note_Site'] = data_gta['Note_Site'].astype(float)

df_gta[["Nb_note_1","Nb_note_2","Nb_note_3","Nb_note_4","Nb_note5_","Nb_note_6","Nb_note_7","Nb_note_8","Nb_note_9","Nb_note_10"]]= data_gta[["Nb_note1","Nb_note2","Nb_note3","Nb_note4","Nb_note5","Nb_note6","Nb_note7","Nb_note8","Nb_note9","Nb_note10"]]

df_gta['link']= data_gta['link']

df_gta.reset_index(drop=True)
df_gta.index = range(1, len(df_gta) + 1)


In [353]:

#df_gta.to_csv('gta_sc.csv', index=False)

df_gta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 1 to 23
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Nom          23 non-null     object 
 1   Genre        23 non-null     object 
 2   Developpeur  23 non-null     object 
 3   Editeur      23 non-null     object 
 4   Date         23 non-null     object 
 5   Console      23 non-null     object 
 6   Note_Site    22 non-null     float64
 7   Nb_note_1    22 non-null     object 
 8   Nb_note_2    22 non-null     object 
 9   Nb_note_3    22 non-null     object 
 10  Nb_note_4    22 non-null     object 
 11  Nb_note5_    22 non-null     object 
 12  Nb_note_6    22 non-null     object 
 13  Nb_note_7    22 non-null     object 
 14  Nb_note_8    22 non-null     object 
 15  Nb_note_9    22 non-null     object 
 16  Nb_note_10   22 non-null     object 
 17  link         23 non-null     object 
dtypes: float64(1), object(17)
memory usage: 3.4+ KB


In [355]:
df_gta.dropna(axis=1)

df_gta.isna().sum()

Nom            0
Genre          0
Developpeur    0
Editeur        0
Date           0
Console        0
Note_Site      1
Nb_note_1      1
Nb_note_2      1
Nb_note_3      1
Nb_note_4      1
Nb_note5_      1
Nb_note_6      1
Nb_note_7      1
Nb_note_8      1
Nb_note_9      1
Nb_note_10     1
link           0
dtype: int64

In [354]:

df_gta.head()
df_gta.dropna(axis=1)

df_gta["Nb_note_1"].apply(lambda x: x[:-1] * 1000 if 'K' in x else int(x))

TypeError: argument of type 'float' is not iterable